In [ ]:
import streamlit as st
import os
import pandas as pd
import json
import mysql.connector

def Agg_insurance():
    path = "C:/Users/Saambavi/OneDrive/Desktop/pulse-master/data/aggregated/insurance/country/india/state/"
    clm = {'State': [], 'Year': [], 'Quarter': [], 'Transaction_count': [], 'Transaction_amount': [],
            'Response_Timestamp': []}

    Agg_state_list = os.listdir(path)
    for i in Agg_state_list:
        p_i = path + i + "/"
        Agg_yr = os.listdir(p_i)
        for j in Agg_yr:
            p_j = p_i + j + "/"
            Agg_yr_list = os.listdir(p_j)
            for k in Agg_yr_list:
                p_k = p_j + k
                with open(p_k, 'r') as Data:
                    D = json.load(Data)
                    response_timestamp = D.get('responseTimestamp')
                    if response_timestamp is not None:
                        for z in D['data']['transactionData']:
                            count = z['paymentInstruments'][0]['count']
                            amount = z['paymentInstruments'][0]['amount']
                            clm['Transaction_count'].append(count)
                            clm['Transaction_amount'].append(amount)
                            clm['State'].append(i)
                            clm['Year'].append(j)
                            clm['Quarter'].append(int(k.strip('.json')))
                            clm['Response_Timestamp'].append(response_timestamp)
                    else:
                        print(f"Missing responseTimestamp in file: {p_k}")
    return pd.DataFrame(clm)

columns3 = Agg_insurance()
columns3["State"] = columns3["State"].str.replace('andaman-&-nicobar-islands', 'Andaman and Nicobar islands')
columns3["State"] = columns3["State"].str.replace("-", " ")
columns3["State"] = columns3["State"].str.title()
columns3["State"] = columns3["State"].str.replace('Dadra & Nagar Haveli & Daman & Diu',
                                                  'Dadra and Nagar Haveli and Daman and Diu')

# Establishing the database connection
db = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='root',
    auth_plugin='mysql_native_password',
    charset='utf8mb4'
)
mycursor = db.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS phonepe_pulse")
mycursor.execute("USE phonepe_pulse")

def Agg_insurance_table():
    # Cdefreating the table if it doesn't exist
    mycursor.execute("""CREATE TABLE IF NOT EXISTS Insurance (
                        State VARCHAR(100),
                        Year INT,
                        Quarter INT,
                        Transaction_count DECIMAL(20,2),
                        Transaction_amount DECIMAL(20,2),
                        Response_Timestamp VARCHAR(255)
                        )""")
    
    # SQL query to insert data
    sql = """INSERT INTO Insurance 
                (State, Year, Quarter, Transaction_count, Transaction_amount, Response_Timestamp)
                VALUES (%s, %s, %s, %s, %s, %s)"""
    
    # Converting DataFrame to list of tuples
    data_list = columns3.values.tolist()
    
    # Inserting each tuple in the list into the table
    for values in data_list:
        mycursor.execute(sql, values)
    
    # Committing the changes
    db.commit()
    # Closing the connection
    db.close()

Agg_insurance_table()


In [ ]:
import streamlit as st
import os
import pandas as pd
import json
import mysql.connector

def Agg_transaction():
    path2 = "C:/Users/Saambavi/OneDrive/Desktop/pulse-master/data/aggregated/transaction/country/india/state/"
    clm11 = {'State': [], 'Year': [], 'Quarter': [], 'name':[] ,'Transaction_count': [], 'Transaction_amount': [],
            'Response_Timestamp': []}

    Agg_state_list = os.listdir(path2)
    for i in Agg_state_list:
        p_i = os.path.join(path2, i)
        Agg_yr = os.listdir(p_i)
        for j in Agg_yr:
            p_j = os.path.join(p_i, j)
            Agg_yr_list = os.listdir(p_j)
            for k in Agg_yr_list:
                p_k = os.path.join(p_j, k)
                with open(p_k, 'r') as Data:
                    D = json.load(Data)
                    response_timestamp = D.get('responseTimestamp')
                    if response_timestamp is not None:
                        for z in D['data']['transactionData']:
                            name1 = z['name']
                            count = z['paymentInstruments'][0]['count']
                            amount = z['paymentInstruments'][0]['amount']
                            clm11['name'].append(name1)
                            clm11['Transaction_count'].append(count)
                            clm11['Transaction_amount'].append(amount)
                            clm11['State'].append(i)
                            clm11['Year'].append(j)
                            clm11['Quarter'].append(int(k.strip('.json')))
                            clm11['Response_Timestamp'].append(response_timestamp)
                    else:
                        print(f"Missing responseTimestamp in file: {p_k}")
    
    df6 = pd.DataFrame(clm11)
    
    df6["State"] = df6["State"].str.replace('andaman-&-nicobar-islands', 'Andaman and Nicobar and Islands')
    df6["State"] = df6["State"].str.replace("-", " ")
    df6["State"] = df6["State"].str.title()
    df6["State"] = df6["State"].str.replace('Dadra & Nagar Haveli & Daman & Diu',
                                            'Dadra and Nagar Haveli and Daman and Diu')
    
    return df6


df6 = Agg_transaction()

# Establish the database connection
db = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='root',
    auth_plugin='mysql_native_password',
    database='phonepe_pulse',
    charset='utf8mb4'
)
mycursor = db.cursor()

def Agg_transaction_table():
    # Create the table if it doesn't exist
    mycursor.execute("""CREATE TABLE IF NOT EXISTS Transaction (
                        State VARCHAR(100),
                        Year INT,
                        Quarter INT,
                        name VARCHAR(100),
                        Transaction_count DECIMAL(20,2),
                        Transaction_amount DECIMAL(20,2),
                        Response_Timestamp VARCHAR(255)
                        )""")
    
    sql = """INSERT INTO Transaction 
                (State, Year, Quarter, name, Transaction_count, Transaction_amount, Response_Timestamp)
                VALUES (%s, %s, %s, %s, %s, %s, %s)"""
    
    # Convert DataFrame to list of tuples
    data_list = df6.values.tolist()
    
    # Insert each tuple in the list into the table
    for values in data_list:
        mycursor.execute(sql, values)
    
    # Commit the changes
    db.commit()
    # Close the connection
    db.close()

Agg_transaction_table()


In [ ]:

path2 = "C:/Users/Saambavi/OneDrive/Desktop/pulse-master/data/aggregated/user/country/india/state/"
         
agg_user_list = os.listdir(path2)

columns2 = {"States":[], "Years":[], "Quarter":[], "Brands":[],"Transaction_count":[], "Percentage":[]}

for state in agg_user_list:
    cur_states = path2+state+"/"
    agg_year_list = os.listdir(cur_states)
    
    for year in agg_year_list:
        cur_years = cur_states+year+"/"
        agg_file_list = os.listdir(cur_years)
        
        for file in agg_file_list:
            cur_files = cur_years+file
            data = open(cur_files,"r")
            C = json.load(data)

            try:

                for i in C["data"]["usersByDevice"]:
                    brand = i["brand"]
                    count = i["count"]
                    percentage = i["percentage"]
                    columns2["Brands"].append(brand)
                    columns2["Transaction_count"].append(count)
                    columns2["Percentage"].append(percentage)
                    columns2["States"].append(state)
                    columns2["Years"].append(year)
                    columns2["Quarter"].append(int(file.strip(".json")))
            
            except:
                pass
aggre_user = pd.DataFrame(columns2)
aggre_user["States"] = aggre_user["States"].str.replace("andaman-&-nicobar-islands","Andaman and Nicobar")
aggre_user["States"] = aggre_user["States"].str.replace("-"," ")
aggre_user["States"] = aggre_user["States"].str.title()
aggre_user['States'] = aggre_user['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")
aggre_user

In [ ]:
import os
import json
import pandas as pd

def map_hover():
    path6 = "C:/Users/Saambavi/OneDrive/Desktop/pulse-master/data/map/insurance/hover/country/india/state"
    columns = {'State': [], 'Year': [], 'Quarter': [], 'Districts': [], 'Transaction_count': [], 'Transaction_amount': [], 'Response_Timestamp': []}

    state_list = os.listdir(path6)
    for state in state_list:
        state_path = os.path.join(path6, state)
        year_list = os.listdir(state_path)
        for year in year_list:
            year_path = os.path.join(state_path, year)
            quarter_list = os.listdir(year_path)
            for quarter_file in quarter_list:
                quarter_path = os.path.join(year_path, quarter_file)
                with open(quarter_path, 'r') as data_file:
                    data = json.load(data_file)
                    response_timestamp = data.get('responseTimestamp')
                    if response_timestamp is not None:
                        for entry in data['data']['hoverDataList']:
                            district = entry['name']
                            count = entry['metric'][0]['count']
                            amount = entry['metric'][0]['amount']
                            columns['Districts'].append(district)
                            columns['Transaction_count'].append(count)
                            columns['Transaction_amount'].append(amount)
                            columns['State'].append(state)
                            columns['Year'].append(year)
                            columns['Quarter'].append(int(quarter_file.strip('.json')))
                            columns['Response_Timestamp'].append(response_timestamp)
                    else:
                        print(f"Missing responseTimestamp in file: {quarter_path}")

    df = pd.DataFrame(columns)
    df["State"] = df["State"].str.replace('andaman-&-nicobar-islands', 'Andaman and Nicobar island')
    df["State"] = df["State"].str.replace("-", " ")
    df["State"] = df["State"].str.title()
    df["State"] = df["State"].str.replace('Dadra & Nagar Haveli & Daman & Diu', 'Dadra and Nagar Haveli and Daman and Diu')

  
    return df

# Call the function
map_hover()


In [ ]:
import os
import json
import pandas as pd

def map_transaction():
    path7= "C:/Users/Saambavi/OneDrive/Desktop/pulse-master/data/map/transaction/hover/country/india/state/"
    columns5 = {'State': [], 'Year': [], 'Quarter': [], 'Districts': [], 'Transaction_count': [], 'Transaction_amount': [], 'Response_Timestamp': []}

    state_list = os.listdir(path7)
    for state in state_list:
        state_path = os.path.join(path7, state)
        year_list = os.listdir(state_path)
        for year in year_list:
            year_path = os.path.join(state_path, year)
            quarter_list = os.listdir(year_path)
            for quarter_file in quarter_list:
                quarter_path = os.path.join(year_path, quarter_file)
                with open(quarter_path, 'r') as data_file:
                    data = json.load(data_file)
                    response_timestamp = data.get('responseTimestamp')
                    if response_timestamp is not None:
                        for entry in data['data']['hoverDataList']:
                            district = entry['name']
                            count = entry['metric'][0]['count']
                            amount = entry['metric'][0]['amount']
                            columns5['Districts'].append(district)
                            columns5['Transaction_count'].append(count)
                            columns5['Transaction_amount'].append(amount)
                            columns5['State'].append(state)
                            columns5['Year'].append(year)
                            columns5['Quarter'].append(int(quarter_file.strip('.json')))
                            columns5['Response_Timestamp'].append(response_timestamp)
                    else:
                        print(f"Missing responseTimestamp in file: {quarter_path}")

    df1 = pd.DataFrame(columns5)
    df1["State"] = df1["State"].str.replace('andaman-&-nicobar-islands', 'Andaman and Nicobar island')
    df1["State"] = df1["State"].str.replace("-", " ")
    df1["State"] = df1["State"].str.title()
    df1["State"] = df1["State"].str.replace('Dadra & Nagar Haveli & Daman & Diu', 'Dadra and Nagar Haveli and Daman and Diu')

    

    return df1
# Call the function
map_transaction()


In [ ]:
import os
import json
import pandas as pd

def map_user():
    path8= "C:/Users/Saambavi/OneDrive/Desktop/pulse-master/data/map/user/hover/country/india/state/"
    column6 = {'State': [], 'Year': [], 'Quarter': [], 'Districts': [], "RegisteredUser":[], "AppOpens":[], 'Response_Timestamp': []}

    state_list = os.listdir(path8)
    for state in state_list:
        state_path = os.path.join(path8, state)
        year_list = os.listdir(state_path)
        for year in year_list:
            year_path = os.path.join(state_path, year)
            quarter_list = os.listdir(year_path)
            for quarter_file in quarter_list:
                quarter_path = os.path.join(year_path, quarter_file)
                with open(quarter_path, 'r') as data_file:
                    data = json.load(data_file)
                    response_timestamp = data.get('responseTimestamp')
                    if response_timestamp is not None:
                        for v in data['data']['hoverData'].items():
                            district = v[0]
                            registereduser = v[1]['registeredUsers']
                            appopens = v[1]['appOpens']
                            column6['Districts'].append(district)
                            column6['RegisteredUser'].append(registereduser)
                            column6['AppOpens'].append(appopens)
                            column6['State'].append(state)
                            column6['Year'].append(year)
                            column6['Quarter'].append(int(quarter_file.strip('.json')))
                            column6['Response_Timestamp'].append(response_timestamp)
                    else:
                        print(f"Missing responseTimestamp in file: {quarter_path}")

    df2 = pd.DataFrame(column6)
    df2["State"] = df2["State"].str.replace('andaman-&-nicobar-islands', 'Andaman and Nicobar island')
    df2["State"] = df2["State"].str.replace("-", " ")
    df2["State"] = df2["State"].str.title()
    df2["State"] = df2["State"].str.replace('Dadra & Nagar Haveli & Daman & Diu', 'Dadra and Nagar Haveli and Daman and Diu')

   
    return df2
# Call the function
df2 = map_user()
print(df2)


In [ ]:
import os
import json
import pandas as pd

def top_insurance():
    path7= "C:/Users/Saambavi/OneDrive/Desktop/pulse-master/data/top/insurance/country/india/state/"
    column8 = {'State': [], 'Year': [], 'Quarter':[], 'Pincodes':[] , 'Transaction_count': [], 'Transaction_amount': [], 'Response_Timestamp': []}

    state_list = os.listdir(path7)
    for state in state_list:
        state_path = os.path.join(path7, state)
        year_list = os.listdir(state_path)
        for year in year_list:
            year_path = os.path.join(state_path, year)
            quarter_list = os.listdir(year_path)
            for quarter_file in quarter_list:
                quarter_path = os.path.join(year_path, quarter_file)
                with open(quarter_path, 'r') as data_file:
                    data = json.load(data_file)
                    response_timestamp = data.get('responseTimestamp')
                    if response_timestamp is not None:
                        for entry in data['data']['pincodes']:
                            entityName = entry['entityName']
                            count = entry['metric']['count']
                            amount = entry['metric']['amount']
                            column8['Pincodes'].append(entityName)
                            column8['Transaction_count'].append(count)
                            column8['Transaction_amount'].append(amount)
                            column8['State'].append(state)
                            column8['Year'].append(year)
                            column8['Quarter'].append(int(quarter_file.strip('.json')))
                            column8['Response_Timestamp'].append(response_timestamp)
                    else:
                        print(f"Missing responseTimestamp in file: {quarter_path}")

    df5= pd.DataFrame(column8)
    df5["State"] = df5["State"].str.replace('andaman-&-nicobar-islands', 'Andaman and Nicobar island')
    df5["State"] = df5["State"].str.replace("-", " ")
    df5["State"] = df5["State"].str.title()
    df5["State"] = df5["State"].str.replace('Dadra & Nagar Haveli & Daman & Diu', 'Dadra and Nagar Haveli and Daman and Diu')

   
    return df5

# Call the function
top_insurance()


In [ ]:
import os
import json
import pandas as pd

def top_transaction():
    path8= "C:/Users/Saambavi/OneDrive/Desktop/pulse-master/data/top/transaction/country/india/state/"
    column8 = {'State': [], 'Year': [], 'Quarter':[], 'Entityname':[] , 'Transaction_count': [], 'Transaction_amount': [], 'Response_Timestamp': []}

    state_list = os.listdir(path8)
    for state in state_list:
        state_path = os.path.join(path8, state)
        year_list = os.listdir(state_path)
        for year in year_list:
            year_path = os.path.join(state_path, year)
            quarter_list = os.listdir(year_path)
            for quarter_file in quarter_list:
                quarter_path = os.path.join(year_path, quarter_file)
                with open(quarter_path, 'r') as data_file:
                    data = json.load(data_file)
                    response_timestamp = data.get('responseTimestamp')
                    if response_timestamp is not None:
                        for entry in data['data']['districts']:
                            entityName = entry['entityName']
                            count = entry['metric']['count']
                            amount = entry['metric']['amount']
                            column8['Entityname'].append(entityName)
                            column8['Transaction_count'].append(count)
                            column8['Transaction_amount'].append(amount)
                            column8['State'].append(state)
                            column8['Year'].append(year)
                            column8['Quarter'].append(int(quarter_file.strip('.json')))
                            column8['Response_Timestamp'].append(response_timestamp)
                    else:
                        print(f"Missing responseTimestamp in file: {quarter_path}")

    df3 = pd.DataFrame(column8)
    df3["State"] = df3["State"].str.replace('andaman-&-nicobar-islands', 'Andaman and Nicobar island')
    df3["State"] = df3["State"].str.replace("-", " ")
    df3["State"] = df3["State"].str.title()
    df3["State"] = df3["State"].str.replace('Dadra & Nagar Haveli & Daman & Diu', 'Dadra and Nagar Haveli and Daman and Diu')

   
    return df3

# Call the function
top_transaction()


In [ ]:
import os
import json
import pandas as pd

def top_user():
    path9= "C:/Users/Saambavi/OneDrive/Desktop/pulse-master/data/top/user/country/india/state/"
    column9 = {'State': [], 'Year': [], 'Quarter':[], 'Districts':[] , 'Registereduser': [], 'Response_Timestamp': []}

    state_list = os.listdir(path9)
    for state in state_list:
        state_path = os.path.join(path9, state)
        year_list = os.listdir(state_path)
        for year in year_list:
            year_path = os.path.join(state_path, year)
            quarter_list = os.listdir(year_path)
            for quarter_file in quarter_list:
                quarter_path = os.path.join(year_path, quarter_file)
                with open(quarter_path, 'r') as data_file:
                    data = json.load(data_file)
                    response_timestamp = data.get('responseTimestamp')
                    if response_timestamp is not None:
                        for entry in data['data']['districts']:
                            Name = entry['name']
                            register = entry['registeredUsers']
                            column9['Districts'].append(Name)
                            column9['Registereduser'].append(register)
                            column9['State'].append(state)
                            column9['Year'].append(year)
                            column9['Quarter'].append(int(quarter_file.strip('.json')))
                            column9['Response_Timestamp'].append(response_timestamp)
                    else:
                        print(f"Missing responseTimestamp in file: {quarter_path}")

    df4 = pd.DataFrame(column9)
    df4["State"] = df4["State"].str.replace('andaman-&-nicobar-islands', 'Andaman and Nicobar island')
    df4["State"] = df4["State"].str.replace("-", " ")
    df4["State"] = df4["State"].str.title()
    df4["State"] = df4["State"].str.replace('Dadra & Nagar Haveli & Daman & Diu', 'Dadra and Nagar Haveli and Daman and Diu')

   
    return df4

# Call the function
top_user()
